# Save test and train data to csv to faster reading

Note: for every pedestrian saves each ten point, e.g.: p1, ..., p10; p2, ..., p11; ...

In [1]:
import pandas as pd

In [2]:
ZERO_PADDING = 6

def get_filename(index):
    return "src/Csv/{}.txt".format(str(index).zfill(ZERO_PADDING))

NUMBER_OF_PEDESTRIANS = 12273

def download_pedestrian(index):
    error_message_template = "Pedestrian number should be between 0 and {max}; given number: {id}"
    assert(0 <= index < NUMBER_OF_PEDESTRIANS), error_message_template.format(max=NUMBER_OF_PEDESTRIANS-1, id=index)
    filename = get_filename(index)
    data = pd.read_csv(filename, index_col=0)
    return data

In [3]:
# in each row: {index; category; frames_number; label}
labels = pd.read_csv('src/pedestrian_labels_and_test_and_train_separation.csv', index_col=0)

In [20]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for index in range(NUMBER_OF_PEDESTRIANS):
    pedestrian_data = pd.DataFrame()
    
    # in each row: {index (time); X; Y}    
    all_data = download_pedestrian(index)[['X', 'Y']]
    
    for i in range(len(all_data) - 9):
        # we want only serial data
        if all_data.index.values[i + 9] - all_data.index.values[i] == 180:
            pedestrian_data = pd.concat([pedestrian_data, all_data[i:i+10].reset_index()[['X', 'Y']]], axis=1)  
    
    if labels['category'][index] == 'train':
        # in each row: {X; Y; X; Y; ...}
        train_data = pd.concat([train_data, pedestrian_data], axis=1)
    else:    
        test_data = pd.concat([test_data, pedestrian_data], axis=1)

In [23]:
test_data.to_csv('src/test_data_points.csv')
train_data.to_csv('src/train_data_points.csv')